## <110>-oriented dumbbell diffusion in Fe-dilute X (X=Cu, Ni, Si) alloy.

In this notebook, we'll create and save a onsager transport coefficient calculator for dumbbell diffusion in Fe-dilute Cu, Ni and Si alloys using all dumbbell diffusion mechanisms in the paper by Messina et. al. at https://doi.org/10.1016/j.actamat.2020.03.038, with the migration barrier data provided in the corresponding database. These mechanism include the 60-degree roto-translation mechanism of <110> dumbbells, as well as rigid translations and on-site rotations, but for the mixed dumbbells, on-site rotations are not considered.

To do this, we need to create a 1nn3 thermodynamic shell (all sites up to the first of first of first nearest neighbors of the solute), large enough to encompass all the solute-dumbbell configurations mentioned in this paper, and also extract out the necessary dumbbell jumps from those created created by our code.

In [1]:
import numpy as np
from onsager.crystal import Crystal
from onsager.OnsagerCalc import dumbbellMediated
from onsager.crystal import DB_disp, DB_disp4, pureDBContainer, mixedDBContainer
from onsager.DB_structs import dumbbell, SdPair, jump, connector

In [3]:
# make a BCC lattice
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.2831
Fe = Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])
# Now give it the orientations - for BCC it's [110], we'll make the dumbbell length same as Fe atomic diameter.
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = pureDBContainer(Fe, 0, family)
mdbcontainer_fe = mixedDBContainer(Fe, 0, family)

# Next, we generate the pure and mixed dumbbell jump networks.
# We set the cutoff jump distance to a little more than the corner-to-body center distance
# jset0 contains pure dumbbell jumps, and jset2 contains mixed dumbbell jumps
# We keep collision thresholds to a small value of 0.01 nm, so that
# as many jumps as possible are initially built.
# We'll extract the jumps we need from those.
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01)

In [4]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to smallest net atomic displacement
def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = DB_disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

indices = []
for jt, jlist in enumerate(jnet0):
    indices.append(jt)

# The roto-translation and rigid translations are the first two jumps
# with the smallest displacement of atoms.
# Then next one is the on-site rotations
# we print these jumps out in the next cell as well.
ind_sort = sorted(indices, key=sortkey)[:3]
len(ind_sort)

3

In [5]:
# Let's print the jumps to see if we have the right jumps
# We'll consider all three types of jumps for pure dumbbell:
# the 60-degree roto-translation, the on-site rotations and
# rigid translations.

# First, we print out the dumbbell orientations
print("Pure dumbbell (i, or) list:")
for tup in pdbcontainer_fe.iorlist:
    print(tup)
    
print()

# For a pure dumbbell, the (i, or) indices (iorInd) correspond to (basis site, orientation vector) pairs.
# The corresponding values can be found in the "iorlist" we just printed out.

# c1 = 1 means the atom at the head of the initial orientation vector jumps.
# c1 = -1 means the atom at the tail of the initial orientation vector jumps.

# c2 = 1 means the atom lands at the head of the final orientation vector.
# c2 = -1 means the atom lands at the tail of the final orientation vector.

print("List of symmetry-unique jumps, sorted according to total distance moved by all atoms:\n")

counter=1
for ind in ind_sort:
    j = jnet0[ind][0]
    
    o1 = pdbcontainer_fe.iorlist[j.state1.iorind][1] * j.c1
    o2 = pdbcontainer_fe.iorlist[j.state2.iorind][1] * j.c2
    n1 = np.linalg.norm(o1)
    n2 = np.linalg.norm(o2)
    
    angle = np.arccos(np.dot(o1, o2) / (n1 * n2)) * 180 / np.pi
    
    print("{}. Symmetry-unique jump type: {}".format(counter, ind))
    print(j)
    print("dumbbell positions and orientiations in Cartesian coordinates:")
    print("initial dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state1.R),", ",
          pdbcontainer_fe.iorlist[j.state1.iorind][1])
    print("Final dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state2.R), ", ",
          pdbcontainer_fe.iorlist[j.state2.iorind][1])
    
    # For the roto-translation, this should be 120, for the on-site rotation 60, and for
    # the rigid translation 180
    print("angle of dumbbell rotation: {:.1f}".format(angle))
    print()
    counter += 1

Pure dumbbell (i, or) list:
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([-0.17819091,  0.        ,  0.17819091]))
(0, array([0.17819091, 0.17819091, 0.        ]))
(0, array([0.        , 0.17819091, 0.17819091]))

List of symmetry-unique jumps, sorted according to total distance moved by all atoms:

1. Symmetry-unique jump type: 8
Jump object:
Initial state:
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 1, lattice vector = [1 0 0]
Jumping from c1 = 1 to c2 = -1

dumbbell positions and orientiations in Cartesian coordinates:
initial dumbbell position and orientiation:  [0. 0. 0.] ,  [0.         0.17819091 0.17819091]
Final dumbbell position and orientiation:  [0.14155 0.14155 0.14155] ,  [0.17819091 0.         0.17819091]
angle of dumbbell rotation: 120.0

2. Symmetry-unique jump type: 17
Jump object:
Initial state:
	dum

In [6]:
# take all the jumps we want.
jset0new = ([jnet0[i] for i in ind_sort], [jnet0_indexed[i] for i in ind_sort])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork.
# We'll include all the jumps there are in the database of Messina et. al.
# They are: 60 deg rot-trans, rigid translation.
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want only the rotational jumps as listed.

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices2 = []
for jt, jlist in enumerate(jnet2):
    # for mixed dumbbells, we will not consider on-site rotations.
    if np.allclose(jnet2_indexed[jt][0][1], z):
        continue
    indices2.append(jt)

# The roto-translation and rigid translations are the first two jumps
# with the smallest displacement of atoms.
ind_sort2 = sorted(indices2, key=sortkey2)[:2]
indices2all = ind_sort2[:2]
print(indices2all)

[15, 16]


In [9]:
# check if we got the correct jumps
# First, we print out the dumbbell orientations
print("Mixed dumbbell (i, or) list:")
for tup in mdbcontainer_fe.iorlist:
    print(tup)
    
print()

# For a mixed dumbbell, the (i, or) indices (iorInd) correspond to (basis site, orientation vector) pairs.
# The corresponding values can be found in the "iorlist" we just printed out.

# For mixed dumbbells, the solute is always considered to be at the head of the dumbbell,
# so c1 and c2 are all 1 for all jumps

print("List of symmetry-unique jumps, sorted according to total distance moved by all atoms:\n")

counter=1
for ind in indices2all:
    j = jnet2[ind][0]
    
    o1 = mdbcontainer_fe.iorlist[j.state1.db.iorind][1] #* j.c1 - this is 1 anyway
    o2 = mdbcontainer_fe.iorlist[j.state2.db.iorind][1] #* j.c2
    n1 = np.linalg.norm(o1)
    n2 = np.linalg.norm(o2)
    
    angle = np.arccos(np.dot(o1, o2) / (n1 * n2)) * 180 / np.pi
    
    print("{}. Symmetry-unique jump type: {}".format(counter, ind))
    print(j)
    print()
    print("Dumbbell positions and orientiations in Cartesian coordinates:")
    print("initial dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state1.db.R),", ",
          mdbcontainer_fe.iorlist[j.state1.db.iorind][1])
    print("Final dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state2.db.R), ", ",
          mdbcontainer_fe.iorlist[j.state2.db.iorind][1])
    
    # For the roto-translation, this should be 120, and for
    # the rigid translation 180
    print("angle of dumbbell rotation: {:.1f}".format(angle))
    print()
    counter += 1

Mixed dumbbell (i, or) list:
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([-0.17819091,  0.        ,  0.17819091]))
(0, array([0.17819091, 0.17819091, 0.        ]))
(0, array([0.        , 0.17819091, 0.17819091]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([ 0.        , -0.17819091, -0.17819091]))
(0, array([ 0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([-0.17819091, -0.17819091,  0.        ]))
(0, array([-0.17819091,  0.        , -0.17819091]))

List of symmetry-unique jumps, sorted according to total distance moved by all atoms:

1. Symmetry-unique jump type: 15
Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 7, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1  0  0]
	dumbbell : (i

In [11]:
# take the jumps we just extracted.
jset2new = ([jnet2[i] for i in indices2all], [jnet2_indexed[i] for i in indices2all])

In [12]:
# Now initialize the onsager calculator.
# Now, we construct the Onsager calculator with the pure and mixed dumbbell jump networks.
# These will be used to build the omega1 jump networks as well.
# We also set the collision threshold from omega43 jumps to 0.01 nm initially, so
# that as many jumps as possible are built, from which we'll extract the necessary
# 60-degree roto-translation jumps. The cutoff distance for the omega43 jumps is also
# set to 0.26 nm, sligthly larger than the nearest neighbor distance in the BCC iron unit cell.
import time
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.26,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=3)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
generating thermodynamic shell
built shell 1: time - 0.0537259578704834
built shell 2: time - 2.111366033554077
built shell 3: time - 9.371391773223877
grouped states by symmetry: 9.584547758102417
built mixed dumbbell stars: 0.0007727146148681641
built jtags2: 0.00011539459228515625
built mixed indexed star: 0.008775711059570312
building star2symlist : 0.0002193450927734375
building bare, mixed index dicts : 0.00034880638122558594
thermodynamic shell generated: 30.679481983184814
Total number of states in Thermodynamic Shell - 546, 12
generating kinetic shell
built shell 1: time - 0.05363011360168457
built shell 2: time - 2.1504127979278564
built shell 3: time - 11.620676279067993
built shell 4: time - 24.757089138031006
grouped states by symmetry: 39.861923694610596
built mixed dumbbell stars: 0.0008187294006347656
built jtags2: 0.00010895729064941406
built mixed indexed star: 0.00908970832824707
building star2symlist : 0.0003924369812011719
build

In [13]:
# Next, we extract the jumps we need for the mixed dumbbell formation and annihilation.
# The association and dissociation occurs via 60-degree roto-translation mechanism.

# We first gather all the jumps generated by the calculator.
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed

def sortkey3(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
ind_sort43 = sorted(indices43, key=sortkey3)[:1]
print(ind_sort43)

[10]


In [14]:
# Let's take a look at the extracted omega 43 jumps.
# Since they have the same transition state, they are all grouped together in
# the same list. Even indices in this list (0, 2, 4...) will give us omega4 jumps.
# The odd indices (1, 3, 5,...) correspond to omega3 jumps.

# For example jnet43[some_list_index][0] is an omega4 jump, while jnet43[some_list_index][1] is an omega3 jump.

# First, we print out the dumbbell (basis site, orientation) list
print("Pure dumbbell (basis site, orientation) or (i, or) list:")
for tup in pdbcontainer_fe.iorlist:
    print(tup)
    
print()

# Then let's print out the mixed dumbbell (basis site, orientation) list
print("Mixed dumbbell (basis site, orientation) or (i, or) list:")
for tup in mdbcontainer_fe.iorlist:
    print(tup)
print()

# Remmber that in an omega4 jump, the self-interstitial jumps, and the solute is always considered
# to be at the head of the dumbbell, so c2 will always be -1 (since the self-interstitial is considered
# to land at the tail of the mixed dumbbell).

j = jnet43[ind_sort43[0]][0] # This is an omega4 jump.
print(j,"\n")
print("dumbbell positions and orientiations in Cartesian coordinates:")
print("initial dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state1.db.R),", ",
      pdbcontainer_fe.iorlist[j.state1.db.iorind][1]) # the initial state is a pure dumbbell
print("Final dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state2.db.R), ", ",
     mdbcontainer_fe.iorlist[j.state2.db.iorind][1])

Pure dumbbell (basis site, orientation) or (i, or) list:
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([-0.17819091,  0.        ,  0.17819091]))
(0, array([0.17819091, 0.17819091, 0.        ]))
(0, array([0.        , 0.17819091, 0.17819091]))

Mixed dumbbell (basis site, orientation) or (i, or) list:
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([-0.17819091,  0.        ,  0.17819091]))
(0, array([0.17819091, 0.17819091, 0.        ]))
(0, array([0.        , 0.17819091, 0.17819091]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([ 0.        , -0.17819091, -0.17819091]))
(0, array([ 0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([-0.17819091, -0.17819091,  0.        ]))
(0, array([-0.1781

In [15]:
# Next, we re-initialize the omega43 jump networks in the onsagercalculator
# with the jumps we need.
onsagercalculator.regenerate43(ind_sort43)

In [16]:
# We now save the calculator to use in our calculations.
import pickle
with open('Fe_Cu_Ni_Si.pkl','wb') as fl:
    pickle.dump(onsagercalculator,fl)